# UniFi API Configuration with 1Password

This notebook demonstrates how to securely store and retrieve UniFi API credentials using **1Password CLI** instead of plain text `~/.env` files.

**Production Setup:**
- 🏦 **Production Vault**: `Beastmaster` (for all Beastmaster project credentials)
- 📦 **Archive Vault**: `Azure-Bot-Config` (backup/archive)
- 🔒 **No secrets in notebook**: All credentials stored securely in 1Password

**Why 1Password?**
- 🔒 **Enterprise-grade security**: Credentials encrypted at rest and in transit
- 🔐 **No plain text files**: Secrets never stored in files on disk
- 🔑 **Biometric authentication**: Leverages your system's secure authentication
- 👥 **Team sharing**: Share secrets securely with team members
- 📱 **Multi-device**: Access from any device with 1Password
- 🔄 **Audit trail**: Track who accessed what and when

---

## Architecture Overview

```mermaid
graph TB
    subgraph "Your Environment"
        Notebook["📓 Jupyter Notebook"]
        PythonCode["🐍 Python Code"]
    end
    
    subgraph "1Password Integration"
        OPCLI["💻 1Password CLI<br/>op command"]
        OPAuth["🔐 Authentication<br/>Browser/Desktop App"]
        OPVault["🗄️ 1Password Vault<br/>Encrypted Storage"]
    end
    
    subgraph "UniFi APIs"
        SiteAPI["☁️ Site Manager API"]
        LocalAPI["🏠 Local Network API"]
    end
    
    Notebook --> PythonCode
    PythonCode -->|"Retrieve Secrets"| OPCLI
    OPCLI -->|"Authenticate"| OPAuth
    OPCLI -->|"Fetch from"| OPVault
    OPVault -->|"Returns Encrypted"| OPCLI
    OPCLI -->|"Secrets"| PythonCode
    PythonCode -->|"API Key"| SiteAPI
    PythonCode -->|"API Token"| LocalAPI
    
    style Notebook fill:#e1f5ff
    style OPVault fill:#fff4e1
    style OPAuth fill:#ffcdd2
    style SiteAPI fill:#e8f5e9
    style LocalAPI fill:#fce4ec
```

---

## Prerequisites

- 1Password account (personal or business)
- 1Password desktop app installed (for CLI integration)
- 1Password CLI (`op`) installed (we'll do this in the notebook)


In [ ]:
# Install required libraries
import sys
!{sys.executable} -m pip install requests python-dotenv matplotlib playwright --quiet

print("✓ Python libraries installed")
print("ℹ Playwright installed for browser automation (optional)")
print("   Run 'python3 -m playwright install chromium' if needed for automation")


## Step 1: Install 1Password CLI

The 1Password CLI (`op`) allows us to interact with 1Password from the command line and retrieve secrets programmatically.


In [ ]:
import subprocess
import shutil
from pathlib import Path

# Check if op is already installed
op_path = shutil.which('op')

if op_path:
    print(f"✓ 1Password CLI already installed at: {op_path}")
    # Get version
    try:
        result = subprocess.run(['op', '--version'], capture_output=True, text=True, timeout=5)
        if result.returncode == 0:
            print(f"✓ Version: {result.stdout.strip()}")
        else:
            print(f"⚠️  CLI found but version check failed")
    except:
        print(f"⚠️  CLI found but unable to check version")
else:
    print("ℹ 1Password CLI not found. Installing...")
    print("\n" + "="*60)
    print("INSTALLATION OPTIONS")
    print("="*60)
    print("\nOption 1: Using Homebrew (macOS)")
    print("  Run in terminal: brew install --cask 1password-cli")
    print("\nOption 2: Manual installation")
    print("  1. Download from: https://1password.com/downloads/command-line/")
    print("  2. Follow installation instructions for your OS")
    print("  3. Verify with: op --version")
    print("\nAfter installation, restart this notebook or run the next cell.")


In [ ]:
# Attempt to install via Homebrew (if available)
import subprocess
import sys

print("="*60)
print("INSTALLING 1PASSWORD CLI")
print("="*60)

# Check if brew is available
brew_path = shutil.which('brew')
if not brew_path:
    print("ℹ Homebrew not found. Please install manually:")
    print("  1. Download from: https://1password.com/downloads/command-line/")
    print("  2. Or install Homebrew first: https://brew.sh")
else:
    print(f"✓ Homebrew found at: {brew_path}")
    print("\nInstalling 1Password CLI...")
    print("(This may open a browser window for authentication)")
    
    try:
        # Run brew install
        result = subprocess.run(
            ['brew', 'install', '--cask', '1password-cli'],
            capture_output=True,
            text=True,
            timeout=300  # 5 minute timeout
        )
        
        if result.returncode == 0:
            print("✓ Installation completed successfully!")
            # Verify installation
            op_path = shutil.which('op')
            if op_path:
                print(f"✓ 1Password CLI verified at: {op_path}")
                version_result = subprocess.run(['op', '--version'], capture_output=True, text=True, timeout=5)
                if version_result.returncode == 0:
                    print(f"✓ Version: {version_result.stdout.strip()}")
        else:
            print("⚠️  Installation may require user interaction")
            print("  Output:", result.stdout[:200])
            if result.stderr:
                print("  Errors:", result.stderr[:200])
    except subprocess.TimeoutExpired:
        print("⚠️  Installation timed out (this is normal - may need manual completion)")
    except Exception as e:
        print(f"⚠️  Installation error: {e}")
        print("\n💡 Try installing manually:")
        print("  brew install --cask 1password-cli")


## Step 2: Enable 1Password CLI Integration

Before using the CLI, you need to enable integration with the 1Password desktop app.

### How to Enable Integration

```mermaid
flowchart TD
    Start([Start]) --> OpenApp[Open 1Password<br/>Desktop App]
    OpenApp --> Unlock[Unlock 1Password]
    Unlock --> Settings[Go to Settings]
    Settings --> Developer[Find Developer<br/>Section]
    Developer --> Enable[Enable<br/>Integrate with<br/>1Password CLI]
    Enable --> Done([✓ Ready])
    
    style Start fill:#e1f5ff
    style Done fill:#c8e6c9
```

**Steps:**
1. Open the **1Password desktop application**
2. Unlock your vault
3. Go to **Settings** → **Developer**
4. Enable **"Integrate with 1Password CLI"**
5. You may be prompted to authenticate


In [ ]:
# Check if 1Password CLI integration is ready
import subprocess
import json

print("="*60)
print("CHECKING 1PASSWORD CLI STATUS")
print("="*60)

op_path = shutil.which('op')
if not op_path:
    print("✗ 1Password CLI not found. Please install it first.")
    print("  See previous cells for installation instructions.")
else:
    print(f"✓ 1Password CLI found at: {op_path}")
    
    # Try to check account status (this will prompt for auth if not signed in)
    try:
        result = subprocess.run(
            ['op', 'account', 'list'],
            capture_output=True,
            text=True,
            timeout=10
        )
        
        if result.returncode == 0 and result.stdout.strip():
            print("✓ CLI is authenticated!")
            print("\nAccount(s):")
            print(result.stdout)
        elif "not signed in" in result.stderr.lower() or "signin" in result.stderr.lower():
            print("ℹ Not signed in yet. We'll sign in next.")
        else:
            print("ℹ Status unclear. Attempting to sign in next...")
    except subprocess.TimeoutExpired:
        print("⚠️  Command timed out")
    except Exception as e:
        print(f"ℹ Status check: {str(e)[:100]}")
        print("  This is normal if you haven't signed in yet.")


## Step 3: Sign In to 1Password CLI

The first time you use the CLI, you'll need to sign in. This will open a browser window where you can authenticate.


In [ ]:
# Optional: Browser automation with Playwright
# This provides full browser automation capabilities (not just opening a browser)
from playwright.sync_api import sync_playwright
import subprocess

print("="*60)
print("BROWSER AUTOMATION OPTION (Playwright)")
print("="*60)

# Check if already signed in first
try:
    result = subprocess.run(['op', 'account', 'list'], capture_output=True, text=True, timeout=5)
    if result.returncode == 0 and result.stdout.strip():
        print("✓ Already signed in to 1Password CLI")
        print("\n💡 Browser automation not needed - you're already authenticated!")
        print("\nCurrent account(s):")
        print(result.stdout)
        automation_needed = False
    else:
        automation_needed = True
except:
    automation_needed = True

if automation_needed:
    print("\n💡 Playwright can automate the sign-in process:")
    print("   - Opens browser with full automation hooks")
    print("   - Can fill forms, click buttons, handle auth")
    print("   - Useful for automated testing or setup")
    print("\n⚠️  Note: 1Password CLI usually handles sign-in automatically")
    print("   Run: op signin")
    print("   This will open browser and handle authentication")
    print("\n💡 To use Playwright automation (if needed):")
    print("   Uncomment the code below and customize for your needs")
    
    # Uncomment to use Playwright automation:
    """
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=False)  # Show browser
        context = browser.new_context()
        page = context.new_page()
        
        # Navigate to 1Password sign-in
        page.goto('https://start.1password.com')
        
        # Your automation code here
        # page.fill('input[name="email"]', 'your-email@example.com')
        # page.click('button[type="submit"]')
        # ... etc
        
        # Keep browser open for manual completion
        input("Press Enter when sign-in is complete...")
        
        browser.close()
    """
else:
    print("\n✅ No automation needed - you're already set up!")


In [ ]:
# Sign in to 1Password CLI
# This will open a browser window for authentication
import subprocess
import webbrowser

print("="*60)
print("SIGNING IN TO 1PASSWORD CLI")
print("="*60)
print("\nThis will open a browser window for authentication.")
print("Please complete the sign-in process in the browser.")
print("\nPress Ctrl+C if you need to cancel, then run this cell again.\n")

try:
    # Check if already signed in
    check_result = subprocess.run(
        ['op', 'account', 'list'],
        capture_output=True,
        text=True,
        timeout=5
    )
    
    if check_result.returncode == 0 and check_result.stdout.strip():
        print("✓ Already signed in!")
        print("\nCurrent account(s):")
        print(check_result.stdout)
        print("\n💡 If you need to re-authenticate or sign in to a different account:")
        print("   Run: op signin")
        print("   Or open browser manually and complete sign-in")
    else:
        print("Initiating sign-in process...")
        print("(A browser window should open shortly)")
        
        # Open browser first
        try:
            print("🌐 Opening browser for sign-in...")
            webbrowser.open('https://start.1password.com')
        except:
            pass
        
        # Sign in - this will prompt for URL/email and open browser
        # For interactive sign-in, we use 'op signin' which opens browser
        signin_result = subprocess.run(
            ['op', 'signin'],
            capture_output=True,
            text=True,
            timeout=60  # Give time for browser authentication
        )
        
        if signin_result.returncode == 0:
            print("✓ Sign-in successful!")
            if signin_result.stdout:
                print(signin_result.stdout)
        else:
            print("⚠️  Sign-in may require manual steps")
            print("\nYou can also sign in manually by running:")
            print("  op signin <account-url>")
            print("  Example: op signin louspringer.1password.com")
            print("\nOr let the interactive prompt guide you.")
            if signin_result.stderr:
                print(f"\nDetails: {signin_result.stderr[:200]}")

except subprocess.TimeoutExpired:
    print("⚠️  Sign-in process timed out")
    print("  This may mean you need to complete authentication in the browser.")
    print("  Please check your browser and complete the sign-in.")
    print("\n💡 Browser should have opened - complete sign-in there")
except KeyboardInterrupt:
    print("\n⚠️  Sign-in cancelled by user")
except Exception as e:
    print(f"⚠️  Sign-in error: {e}")
    print("\n💡 Try signing in manually:")
    print("  op signin")
    print("\n💡 Or open browser manually:")
    print("  python3 -c \"import webbrowser; webbrowser.open('https://start.1password.com')\"")


## Production Setup: Beastmaster Vault

**Note:** This notebook uses the **Beastmaster** vault as the production vault for all Beastmaster project credentials. This vault was created specifically for this lab/work project to keep credentials organized.

### Vault Structure

- **Production Vault**: `Beastmaster`
  - Active credentials for Beastmaster project
  - UniFi API keys and tokens
  - All secure tokens for this work
  
- **Archive Vault**: `Azure-Bot-Config`
  - Original credentials (archived for backup)
  - Preserved but hidden from normal view

### Current Items in Beastmaster Vault

As of setup completion:
- ✓ UniFi Site Manager API Key
- ✓ UniFi Username
- ✓ UniFi Password
- (Local API Token can be added when needed)


In [ ]:
# List available vaults to choose where to store credentials
import subprocess
import json

print("="*60)
print("AVAILABLE 1PASSWORD VAULTS")
print("="*60)

try:
    result = subprocess.run(
        ['op', 'vault', 'list', '--format', 'json'],
        capture_output=True,
        text=True,
        timeout=10
    )
    
    if result.returncode == 0:
        vaults = json.loads(result.stdout)
        if vaults:
            print(f"\n✓ Found {len(vaults)} vault(s):\n")
            for i, vault in enumerate(vaults, 1):
                print(f"{i}. {vault.get('name', 'Unknown')}")
                print(f"   ID: {vault.get('id', 'N/A')}")
                print()
            
            # Store vault info for later use
            if len(vaults) > 0:
                default_vault = vaults[0]['name']
                print(f"💡 We'll use '{default_vault}' as the default vault.")
                print(f"   Modify the vault name in the next cell if needed.\n")
        else:
            print("⚠️  No vaults found")
    else:
        print(f"⚠️  Error listing vaults: {result.stderr}")
        print("  Make sure you're signed in (see previous cell)")
        
except json.JSONDecodeError:
    print("⚠️  Could not parse vault list")
    print(f"  Output: {result.stdout[:200]}")
except Exception as e:
    print(f"⚠️  Error: {e}")
    print("  Make sure you're signed in and 1Password CLI integration is enabled.")


In [ ]:
# Create 1Password items for UniFi credentials
# MODIFY THIS CELL: Set your vault name and credential values

import subprocess
import json

print("="*60)
print("STORING CREDENTIALS IN 1PASSWORD")
print("="*60)

# Configuration - Production Vault Setup
# NOTE: Actual credentials are already stored in the Beastmaster vault.
# This cell shows the structure - if you need to update, modify values below.

VAULT_NAME = "Beastmaster"  # Production vault for Beastmaster project

# Example structure - actual values are stored securely in 1Password vault
# To update credentials, replace the placeholders below with actual values
CREDENTIALS = {
    "UniFi Site Manager API Key": {
        "api_key": "YOUR_API_KEY_HERE"  # Replace with actual value to update, or leave as-is
    },
    "UniFi Local API Token": {
        "api_token": "YOUR_API_TOKEN_HERE"  # Replace with actual value to add/update
    },
    "UniFi Username": {
        "username": "user@example.com"  # Replace with actual value to update
    },
    "UniFi Password": {
        "password": "your_password_here"  # Replace with actual value to update
    }
}

print(f"✓ Using production vault: '{VAULT_NAME}'")
print("\n💡 Note: Actual credentials are already stored in this vault.")
print("   To update credentials, replace placeholder values above with actual values.")
print("   Or leave placeholders as-is to keep existing values.\n")

# Check which items already exist
existing_items = []
for item_name in CREDENTIALS.keys():
    try:
        check_result = subprocess.run(
            ['op', 'item', 'get', item_name, '--vault', VAULT_NAME, '--format', 'json'],
            capture_output=True,
            text=True,
            timeout=5
        )
        if check_result.returncode == 0:
            existing_items.append(item_name)
    except:
        pass

if existing_items:
    print(f"ℹ Found existing items: {', '.join(existing_items)}")
    print("  These will be updated if you provide new values.\n")

# Create or update each credential
created_items = []
updated_items = []

for item_name, fields in CREDENTIALS.items():
    # Check if item value is placeholder
    field_values = list(fields.values())
    if any(val.startswith('YOUR_') or val in ['your_password_here', 'user@example.com'] for val in field_values):
        print(f"⏭️  Skipping '{item_name}' - placeholder value detected")
        if item_name in existing_items:
            print(f"   ℹ Item already exists in '{VAULT_NAME}' vault with actual value")
            print(f"      Keeping existing value - no update needed")
        else:
            print(f"   💡 To add this credential, replace placeholder with actual value above")
        continue
    
    try:
        if item_name in existing_items:
            # Update existing item
            print(f"🔄 Updating existing item: {item_name}")
            
            # Build field update commands
            for field_name, field_value in fields.items():
                update_result = subprocess.run(
                    ['op', 'item', 'edit', item_name, '--vault', VAULT_NAME,
                     f'{field_name}={field_value}'],
                    capture_output=True,
                    text=True,
                    timeout=10
                )
                if update_result.returncode == 0:
                    print(f"   ✓ Updated field: {field_name}")
                else:
                    print(f"   ⚠️  Could not update {field_name}: {update_result.stderr[:100]}")
            
            updated_items.append(item_name)
        else:
            # Create new item
            print(f"➕ Creating new item: {item_name}")
            
            # Create item with fields
            field_args = []
            for field_name, field_value in fields.items():
                field_args.extend([f'{field_name}={field_value}'])
            
            create_result = subprocess.run(
                ['op', 'item', 'create', '--title', item_name, '--vault', VAULT_NAME,
                 '--category', 'Secure Note', '--tags', 'UniFi,API'] + field_args,
                capture_output=True,
                text=True,
                timeout=10
            )
            
            if create_result.returncode == 0:
                print(f"   ✓ Created successfully")
                created_items.append(item_name)
            else:
                print(f"   ✗ Creation failed: {create_result.stderr[:100]}")
                
    except Exception as e:
        print(f"✗ Error processing '{item_name}': {e}")

print(f"\n✓ Summary:")
print(f"   Created: {len(created_items)} items")
print(f"   Updated: {len(updated_items)} items")
print(f"\n💡 To manually create items:")
print(f"   op item create --title 'Item Name' --vault '{VAULT_NAME}' --category 'Secure Note' field_name=value")


## Step 5: Retrieve Credentials from 1Password

Now we'll retrieve the stored credentials and use them for UniFi API authentication.


In [ ]:
# Retrieve credentials from 1Password
import subprocess
import os

print("="*60)
print("RETRIEVING CREDENTIALS FROM 1PASSWORD")
print("="*60)

VAULT_NAME = "Beastmaster"  # Change if you used a different vault

# Item names (must match what you created)
ITEMS = {
    'UNIFI_API_KEY': 'UniFi Site Manager API Key',
    'UNIFI_LOCAL_TOKEN': 'UniFi Local API Token',
    'UNIFI_USERNAME': 'UniFi Username',
    'UNIFI_PASSWORD': 'UniFi Password',
}

# Field names in 1Password items
FIELD_NAMES = {
    'UNIFI_API_KEY': 'api_key',
    'UNIFI_LOCAL_TOKEN': 'api_token',
    'UNIFI_USERNAME': 'username',
    'UNIFI_PASSWORD': 'password',
}

retrieved_credentials = {}

for env_var, item_name in ITEMS.items():
    field_name = FIELD_NAMES[env_var]
    
    try:
        result = subprocess.run(
            ['op', 'item', 'get', item_name, '--vault', VAULT_NAME,
             '--fields', field_name, '--format', 'json'],
            capture_output=True,
            text=True,
            timeout=10
        )
        
        if result.returncode == 0:
            # Parse the field value
            try:
                field_data = json.loads(result.stdout)
                # Handle different response formats
                if isinstance(field_data, list) and len(field_data) > 0:
                    value = field_data[0].get('value', '')
                elif isinstance(field_data, dict):
                    value = field_data.get('value', '') or field_data.get(field_name, '')
                else:
                    # Sometimes it's just the raw value
                    value = result.stdout.strip()
                
                if value:
                    retrieved_credentials[env_var] = value
                    # Show partial value for confirmation
                    preview = value[:10] + '...' if len(value) > 10 else value
                    if env_var == 'UNIFI_PASSWORD':
                        preview = '••••••••'
                    print(f"✓ {env_var:20s}: {preview}")
                else:
                    print(f"⚠️  {env_var:20s}: Field '{field_name}' not found or empty")
            except json.JSONDecodeError:
                # Sometimes op returns plain text
                value = result.stdout.strip()
                if value:
                    retrieved_credentials[env_var] = value
                    preview = value[:10] + '...' if len(value) > 10 else value
                    if env_var == 'UNIFI_PASSWORD':
                        preview = '••••••••'
                    print(f"✓ {env_var:20s}: {preview}")
        else:
            print(f"✗ {env_var:20s}: Item '{item_name}' not found")
            if result.stderr:
                print(f"     Error: {result.stderr[:100]}")
                
    except subprocess.TimeoutExpired:
        print(f"✗ {env_var:20s}: Request timed out")
    except Exception as e:
        print(f"✗ {env_var:20s}: Error - {str(e)[:50]}")

print(f"\n✓ Retrieved {len(retrieved_credentials)} credential(s)")
print(f"  Available for use: {list(retrieved_credentials.keys())}")

# Store in environment for this session
for key, value in retrieved_credentials.items():
    os.environ[key] = value


In [ ]:
# Test UniFi API connections using credentials from 1Password
import requests

print("="*60)
print("TESTING UNIFI API CONNECTIONS")
print("="*60)

# Test Site Manager API
api_key = retrieved_credentials.get('UNIFI_API_KEY')
if api_key:
    print("\n📡 Testing Site Manager API...")
    api_session = requests.Session()
    api_session.headers.update({
        'X-API-Key': api_key,
        'Accept': 'application/json',
    })
    
    try:
        resp = api_session.get('https://api.ui.com/v1/hosts', timeout=15)
        if resp.status_code == 200:
            data = resp.json()
            if 'data' in data:
                host_count = len(data['data']) if isinstance(data['data'], list) else 0
                print(f"   ✓ Connected successfully!")
                print(f"   ✓ Found {host_count} host(s)")
            else:
                print(f"   ✓ Connected (unexpected format)")
        elif resp.status_code == 401:
            print(f"   ✗ Authentication failed - invalid API key")
        else:
            print(f"   ✗ Error {resp.status_code}")
    except Exception as e:
        print(f"   ✗ Connection error: {str(e)[:50]}")
else:
    print("\n⏭️  Skipping Site Manager API test - API key not found")

# Test Local API (if token available)
local_token = retrieved_credentials.get('UNIFI_LOCAL_TOKEN')
if local_token:
    print("\n📡 Testing Local Network API...")
    print("   (This requires the local controller to be accessible)")
    
    # Try to connect to local controller
    local_ips = ['192.168.1.1', '10.0.0.1']
    connected = False
    
    for ip in local_ips:
        try:
            session = requests.Session()
            session.verify = False
            session.headers.update({
                'Authorization': f'Bearer {local_token}',
                'Content-Type': 'application/json'
            })
            
            test_url = f"https://{ip}:443/proxy/network/api/self/sites"
            resp = session.get(test_url, timeout=10)
            
            if resp.status_code == 200:
                print(f"   ✓ Connected to {ip}")
                print(f"   ✓ Local API token is valid")
                connected = True
                break
        except:
            continue
    
    if not connected:
        print("   ⚠️  Could not connect (controller may not be accessible or token invalid)")
else:
    print("\n⏭️  Skipping Local API test - API token not found")

print("\n" + "="*60)
print("✓ Credential testing complete")


## Step 6: Create Helper Function for Future Use

Create a reusable function to load credentials from 1Password that you can use in other notebooks.


In [ ]:
# Create a reusable function for loading credentials from 1Password
import subprocess
import os
import json

def load_unifi_credentials_from_1password(vault_name="Beastmaster"):
    """
    Load UniFi API credentials from 1Password.
    
    Parameters:
    -----------
    vault_name : str
        Name of the 1Password vault containing the credentials
        
    Returns:
    --------
    dict : Dictionary with credential keys (UNIFI_API_KEY, UNIFI_LOCAL_TOKEN, etc.)
    """
    
    ITEMS = {
        'UNIFI_API_KEY': ('UniFi Site Manager API Key', 'api_key'),
        'UNIFI_LOCAL_TOKEN': ('UniFi Local API Token', 'api_token'),
        'UNIFI_USERNAME': ('UniFi Username', 'username'),
        'UNIFI_PASSWORD': ('UniFi Password', 'password'),
    }
    
    credentials = {}
    
    for env_var, (item_name, field_name) in ITEMS.items():
        try:
            result = subprocess.run(
                ['op', 'item', 'get', item_name, '--vault', vault_name,
                 '--fields', field_name, '--format', 'json'],
                capture_output=True,
                text=True,
                timeout=10
            )
            
            if result.returncode == 0:
                try:
                    field_data = json.loads(result.stdout)
                    if isinstance(field_data, list) and len(field_data) > 0:
                        value = field_data[0].get('value', '')
                    elif isinstance(field_data, dict):
                        value = field_data.get('value', '') or field_data.get(field_name, '')
                    else:
                        value = result.stdout.strip()
                    
                    if value:
                        credentials[env_var] = value
                        os.environ[env_var] = value
                except json.JSONDecodeError:
                    value = result.stdout.strip()
                    if value:
                        credentials[env_var] = value
                        os.environ[env_var] = value
        except:
            pass
    
    return credentials

# Example usage
print("="*60)
print("HELPER FUNCTION CREATED")
print("="*60)
print("""
You can now use this function in other notebooks:

```python
from unifi_1password_configuration import load_unifi_credentials_from_1password

# Load credentials from 1Password
creds = load_unifi_credentials_from_1password(vault_name="Beastmaster")

# Credentials are now in os.environ and returned dict
api_key = os.getenv('UNIFI_API_KEY')
# or
api_key = creds.get('UNIFI_API_KEY')
```

To save this function for reuse, you could:
1. Copy it to a separate Python module (.py file)
2. Import it in your other notebooks
3. Or just copy the function code into other notebooks
""")

# Test the function
print("\n" + "="*60)
print("TESTING HELPER FUNCTION")
print("="*60)
test_creds = load_unifi_credentials_from_1password(VAULT_NAME if 'VAULT_NAME' in globals() else "Private")
print(f"✓ Loaded {len(test_creds)} credential(s)")
print(f"  Keys: {list(test_creds.keys())}")


## Production Setup Summary

This section documents the actual production setup used for the Beastmaster project.

### What Was Done

1. ✅ **Created "Beastmaster" vault** - Production vault for Beastmaster project credentials
2. ✅ **Migrated credentials** - Moved UniFi credentials from archive vault to Beastmaster
3. ✅ **Archived originals** - Archived items in original vault for backup
4. ✅ **Updated notebook** - Configured to use Beastmaster vault by default

### Vault Status

**Production: `Beastmaster`**
- Active vault for all Beastmaster work
- Contains: UniFi API Key, Username, Password
- Ready for: Local API Token (when needed)

**Archive: `Azure-Bot-Config`**
- Original credentials archived for backup
- Items preserved but hidden from normal view

### Security Notes

- ✅ **No secrets in notebook** - All actual credentials stored in 1Password
- ✅ **Example values only** - This notebook shows structure, not actual secrets
- ✅ **Lab environment** - Home network setup, not production infrastructure
- ✅ **Secure storage** - All tokens/keys encrypted in 1Password vault


## Launch Notebook in Browser for Better Visualization & Debugging

This cell uses Playwright to launch the notebook in a browser with **full automation hooks**, making it much better for:
- 📊 **Viewing visualizations** clearly (Mermaid diagrams, Matplotlib charts)
- 🐛 **Debugging** with interactive browser tools
- 🎯 **Stepping through cells** interactively
- 🔧 **Meta-documentation** - using the notebook itself as a debugging tool

**Why use this?**
The IDE's notebook viewer is fine for coding, but **horrible for visualizations**. Launching in a browser with Playwright gives you:
- Full browser developer tools
- Better rendering of diagrams
- Full automation hooks for debugging
- Clearer visualization display


In [ ]:
# Launch notebook in browser using Playwright for better visualization & debugging
# This is the "meta-document" debugging tool - using the notebook itself for debugging!

from playwright.sync_api import sync_playwright
import subprocess
import time
import sys
from pathlib import Path

print("="*60)
print("LAUNCHING NOTEBOOK IN BROWSER FOR DEBUGGING")
print("="*60)

notebook_file = Path("unifi_1password_configuration.ipynb")
notebook_name = notebook_file.name

if not notebook_file.exists():
    print(f"✗ Notebook not found: {notebook_file}")
else:
    print(f"✓ Found notebook: {notebook_name}")
    
    # Check if Jupyter server is running
    jupyter_url = None
    
    try:
        import requests
        for port in [8888, 8889, 8890, 8887]:
            try:
                response = requests.get(f'http://localhost:{port}', timeout=2)
                if response.status_code == 200:
                    jupyter_url = f"http://localhost:{port}"
                    print(f"✓ Jupyter server already running on {jupyter_url}")
                    break
            except:
                pass
    except ImportError:
        print("ℹ requests not available - will start new server")
    
    if not jupyter_url:
        print("\n🚀 Starting Jupyter server...")
        print("   (This may take a moment)")
        
        # Start Jupyter server
        jupyter_cmd = [sys.executable, '-m', 'jupyter', 'notebook', '--no-browser', '--port=8888']
        jupyter_process = subprocess.Popen(
            jupyter_cmd,
            cwd=notebook_file.parent,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        
        # Wait for server to start
        print("   Waiting for server to start...")
        for i in range(15):
            time.sleep(1)
            try:
                import requests
                response = requests.get('http://localhost:8888', timeout=1)
                if response.status_code == 200:
                    jupyter_url = "http://localhost:8888"
                    print(f"✓ Server started on {jupyter_url}")
                    break
            except:
                if i % 3 == 0:
                    print(f"   Still waiting... ({i+1}/15)")
        
        if not jupyter_url:
            print("⚠️  Server may need more time - trying anyway...")
            jupyter_url = "http://localhost:8888"
            time.sleep(2)  # Give it a bit more time
    
    if jupyter_url:
        notebook_url = f"{jupyter_url}/notebooks/{notebook_name}"
        print(f"\n🌐 Launching notebook with Playwright browser automation...")
        print(f"   URL: {notebook_url}\n")
        
        try:
            with sync_playwright() as p:
                # Launch browser with full automation hooks
                print("🚀 Launching Chromium browser with automation hooks...")
                browser = p.chromium.launch(
                    headless=False,
                    args=['--start-maximized']
                )
                
                # Create context with large viewport for better visualization
                context = browser.new_context(viewport={'width': 1920, 'height': 1080})
                page = context.new_page()
                
                # Navigate to notebook
                print("📓 Loading notebook...")
                page.goto(notebook_url, wait_until='networkidle', timeout=30000)
                
                print("✓ Notebook opened in browser with full automation hooks!")
                print("\n✅ Browser window is now open - you can:")
                print("   📊 View all visualizations clearly (Mermaid, Matplotlib)")
                print("   🐛 Debug with browser developer tools")
                print("   🎯 Step through cells interactively")
                print("   🔧 Use Playwright hooks for automation")
                print(f"\n📝 Server: {jupyter_url}")
                print(f"🌐 Notebook: {notebook_url}")
                print("\n💡 Browser will stay open - interact with it freely")
                print("   Close the browser window when you're done")
                print("\n⏸️  This is your debugging meta-document!")
                print("   Use it to step through and debug the notebook interactively\n")
                
                # Keep browser open for interaction
                # Browser stays alive until user closes window or interrupts
                try:
                    # Check periodically if browser is still connected
                    while browser.connected:
                        time.sleep(1)
                except KeyboardInterrupt:
                    print("\n\n⚠️  Interrupted - closing browser...")
                
                if browser.connected:
                    browser.close()
                print("✓ Browser closed")
                
        except Exception as e:
            print(f"⚠️  Playwright error: {e}")
            print(f"\n💡 Opening with regular browser instead...")
            import webbrowser
            webbrowser.open(notebook_url)
            print(f"✓ Notebook opened in default browser")
    else:
        print("⚠️  Could not start Jupyter server")
        print(f"   Try manually: {sys.executable} -m jupyter notebook")
